<a href="https://colab.research.google.com/github/kadeng/pykurs/blob/main/notebooks/python_kurs_3_tabellen_transformieren.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Kurs

Dieses Notebook ist Teil 3, eines kleinen Kurses mit Übungen, in dem auf einige Themen mittleren bis einfachen Komplexitätsgrads eingegangen wird, die in generellen Einführungen zu Python manchmal zu kurz kommen, in der Praxis aber relevant sind.


## Tabellen und Tabellen Transformationen

das Thema in diesem Notebook sind Tabellen und wie man sie transformiert. Wir werden dazu das **pandas** Paket benutzen, welches man ggf. installieren muss, sowie das **re** und das **json** Paket, welche zu Python gehören.

Da dieses Notebook ohne weitere zugehörige Dateien lauffähig sein soll, verwenden wir zudem das io Package um CSV Dateien aus im Notebook eingebetteten Strings zu lesen.


In [2]:
import re
import json
import pandas as pd
import io
import numpy as np

In [3]:
# Display optionen setzen
# wir wollen die Daten komplett sehen
pd.set_option('display.max_columns', 100)
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 1000

### Pandas Grundlagen

pandas ist das beliebteste Paket um in Python mit Tabellen zu arbeiten. Alternative und durchaus erwähnenswerte Pakete sind noch **polars** welches etwas neuer und schneller ist. Vermutlich ist **polars** inzwischen besser, aber für **pandas** gibt es mehr Dokumentation und Hilfestellungen Online.



## pandas DataFrame und Series

Die zwei wichtigsten Datentypen die man in Pandas auch vom Namen her kennen sollte sind

### DataFrame ( Tabellen )

 Ein DataFrame ist eine komplette Tabelle, die Zeilen und Spalten hat. Da pandas allerdings die Daten **Spaltenorientiert** speichert, ist effektiv ein Dataframe etwas das einem **dict** ähnelt das einfach nur Spaltennamen (Strings) auf Objekte mappt die Spalten repräsentieren ( **pandas.Series** ).

neben diesen Spalten, verwaltet ein DataFrame auch noch einen **Index** mit dem einzelne Einträge in allen Spalten gefunden werden können. Dieser **Index** muss bei allen Spalten des Dataframes der gleiche sein, das heisst z.B. alle Spalten müssen die gleiche Länge haben. Standardmässig werden die Einträge in einem Dataframe oder auch einzelnen Spalten einfach von 0 aufsteigend durchnumeriert. Aber ein Index kann z.B. auch erlauben, jeder Zeile einen Namen zu geben.

**Man kann einem Dataframe ohne weiteres neue Spalten hinzufügen, Spalten löschen oder ersetze**n. Es ist allerdings **nicht ohne weiteres möglich, neue Zeilen hinzuzufügen oder Zeilen zu ersetzen**. Dazu schreibt man dann in der Regel besser einen neuen DataFrame. Das ist glücklicherweise recht einfach.

 ### Series ( Spalten / Columns )

 Eine pandas **Series repräsentiert in der Regel eine Spalte**. Allgemein ist es aber einfach eine Liste von Werten eines gemeinsamen Datentyps. Zusätzlich zu dieser Liste von Werten hat jede pandas Series noch einen **Index** der in der Regel von 0 aufsteigend numeriert ist.

 Es ist theoretisch möglich einzelne Werte in Spalten zu ändern, allerdings ist es auch da aus Effizienzgründen ratsamer lieber neue Spalten zu erzeugen und die alten Spalten komplett zu ersetzen.

#### Achtung: Zugriff auf Zeilen ( rows )

 Wenn man allerdings aus einem pandas DataFrame auf eine einzelne Zeile ( z.B. mit Hilfe von **DataFrame.rows**) zugreift, so wird auch diese Zeile als ein pandas.Series Objekt repräsentiert, mit einem Index der erlaubt auf einzelne Einträge anhand des Namens der Spalte zuzugreifen. Allerdings wird dieses Series Objekt dann kurzerhand erzeugt und ist eigentlich nur ein temporäres Objekt. Es **macht daher keinen Sinn zu versuchen auf diese Art Zeilen zu verändern**.



### Eine Problematische Tabelle

Wir beginnen mit einer sehr problematischen Tabelle, die aus einer Datenbank, Excel oder sonstwo kommen könnte. Sie hat zahlreiche Probleme die wir nacheinander angehen werden.

Wie man auch in diesem Beispiel gut sieht, wird der pandas DataFrame als
ein dict

In [11]:
daten_column_format =  {  "product_id" : [ 1.0, 2.0, 3.0, 4.0 ], # produkt-id sollte eigentlich ein int sein, sind aber floats
        "price" : [ "100,000.5 $", "1.130,5 EUR", "129.200€", "N/A. Bert fragt nach." ], # Wirrwarr an Formaten und Währungen, mit fehlenden Daten. Sollte eigentlich ein float-Wert in EUR sein
        "tested" : [ "Ja", "Nö", "Nein", "Vielleicht"], # Sollte eigentlich True, False oder None (unbekannt) sein
        "category" : [ "Gartenmoebel", "Gartenmoebel", "Möbel", "Moebel"], # Umlaute mal so mal so, sollte Gruppierungen erlauben
        "belastungstest_stufen" : [ "10,20,30", "10,20,30", "10,20", "10,20"], # kg mit denen die Möbel bei Tests belastet wurden
        "belastungstest_ergebnisse" : [ # Ergebnisse von Belastungstests in einem Freitextformat das man parsen muss
              "ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",
              "nicht OK:  10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",
              "Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",
              "ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok"]}

problem_df = pd.DataFrame(daten_column_format)
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse
0,1.0,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden."
1,2.0,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet"
2,3.0,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität"
3,4.0,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok"


### Spalten Datentyp konvertieren

in diesem Beispiel gibt es die Spalte "product_id", welche eigentlich ein Integer sein sollte. Mit Hilfe der **astype** Methode kann man sie in einen anderen Datentyp konvertieren, sofern das möglich ist (z.B. int zu float oder float zu int ist unproblematisch, aber str zu float muss nicht klappen ).

Dabei muss das Zielformat als ein **numpy** (importiert als "import numpy as np" ) Datentyp angegeben werden. Solche Datentypen sind z.B.

 * np.float32 und np.float64 für 32- und 64-bit floating point Zahlen
 * np.int32 und np.int64 für 32- und 64-bit integer
 * np.bool für boolean ( True/False)
 * np.object für beliebige Objekte, unter anderem für Strings

Zudem hat numpy auch Datentypen für Zeitstempel ( np.datetime64 etc.) die sind aber kompliziert da sie zusätzlich noch eine Einheit benötigen.

In [5]:
# die "astype" Methode erlaubt es uns in
problem_df.product_id.astype(np.int64)

0    1
1    2
2    3
3    4
Name: product_id, dtype: int64

Das konvertieren funktioniert. Jetzt können wir die neue Spalte anstelle der alten Spalte in den Dataframe schreiben

In [6]:
problem_df['product_id'] = problem_df.product_id.astype(np.int64)
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden."
1,2,1.130.5 EUR,Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet"
2,3,129.2€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität"
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok"


### Spalte mit Hilfe von Funktion transformieren

 da wir also in der Regel entweder Zeilen oder ganze Tabellen neu schreiben, tendiert man in pandas dazu, nicht mit Hilfe von "for" loops über die Zeilen oder Spalten zu iterieren, sondern **Funktionen** auf ganze **Zeilen** oder **Einzelne Gruppen von Zeilen** anzuwenden.

als nächstes würden wir gerne die **price** Spalte normalisieren, also in eine "price_eur" spalte überführen die entweder eine float-Zahl oder den float-Wert "nan" (not-a-number) beinhalten darf.

Da in diesem Fall keine einfache Textersetzung ausreicht ( wir müssen z.B. manchmal USD in EUR konvertieren, Komma- oder Punktformat detektieren etc.) müssen wir zunächst lernen wie man eine Spalte Zelle für Zelle durch eine beliebige Python-Funktion transformieren lassen kann. Das ist glücklicherweise nicht schwer mit Hilfe von [pandas.Series.apply](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) und [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)

**Series.apply** erlaubt es, eine normale Python Funktion auf jeden Wert in einer Series anzuwenden und aus dem resultat eine neue Spalte (mit gleichem Index ) zu erzeugen.

Fangen wir mal einfach an, indem wir "€" durch "EUR" ersetzen und "$" durch "USD"

In [8]:
# ersetze $ oder € in price
def normalize_currency_names(s):
  if "$" in s:
    s = s.replace("$", "USD")
  if "€" in s:
    s = s.replace("€", "EUR")
  return s


In [10]:
normalize_currency_names("Test das sind viele $ oder €")

'Test das sind viele USD oder EUR'

In [9]:
# Auf Spalte anwenden
problem_df.price.apply(normalize_currency_names)

0            100,000.5 USD
1              1.130.5 EUR
2                 129.2EUR
3    N/A. Bert fragt nach.
Name: price, dtype: object

#### Übung

wie schreibe ich diese Spalte jetzt zurück in den Dataframe?

### Kommaformat normalisieren

Als nächstes Problem in der Preisspalte ist da die Sache mit Komma oder Punkt. Irgendjemand hat lustigerweise manchmal Komma und manchmal Punkt als Dezimalseparator verwendet, und dann noch um es ja nicht zu einfach zu machen, auch noch Tausender-Trennzeichen verwendet die ein einfaches suchen und ersetzen im Text komplett unmöglich machen. (Ach, das war ja ich.. naja, egal)..



In [15]:
# Versuchen wir das mal anhand von drei Beispielen
beispiele = list(problem_df.price[:-1])
# beachte, dass beim leztzen Fall *gar kein* Kommaseparator benutzt wird, sondern nur ein Tausender-Trennzeichen
beispiele

['100,000.5 $', '1.130,5 EUR', '129.200€']

### Ein Fall für zwei Regular Expressions

Wir versuchen mal das ganze in Regeln zu fassen:

 * Ein Dezimalseparator ist immer das letzte Trennzeichen, **und wird gefolgt von maximal zwei Zahlen**
 * Ein Tausender-Trennzeichen wird gefolgt von genau 3 Ziffern
 * Wir könnten je einen Regulären Ausdruck für Komma-Dezimalseparator und Punkt als Dezimalseparator definieren

 Dann machen wir das mal









In [25]:
# beliebig viele Ziffern, gefolgt von beliebig vielen WIederholungen von .ddd (wobei d für eine beliebige Ziffer steht), gefolt von einem *optionalen* Teil der mit einem Komma beginnt, und danach entweder eine oder zwei Ziffern folgen lässt
komma_muster = r"([0-9]+)(\.[0-9]{3,3})*(\,[0-9]{1,2})?"
re.search(komma_muster, '100,000.5 $')

<re.Match object; span=(0, 6), match='100,00'>

Mist, das hat nicht ganz hingehauen. Vielleicht müssen wir vorher alle Zeichen entfernen die keine Zahlen, Punkte oder Kommas sind, und können danach re.fullmatch verwenden. Oder wir sagen dem Regex, dass **vor und nach** dem Match keine Zahl, Komma oder Punkt kommen darf.

### Übung

implementiere eine Regex Ersetzung, die alle Zeichen aus einem Text entfernt die keine Zahlen, Punkte oder Kommas sind. Verwende dazu die Funktion **re.sub** und eine **invertierte Charakterklasse** im Regex Ausdruck. Beide sind im vorigen Notebook dieses Kurses gut beschrieben. Beachte dass man ggf. auch in Charakterklassen bestimmte Zeichen **escapen** muss.

### Regex Lookahead / Lookbehind

man kann glücklicherweise tatsächlich einer Regex sagen, dass er nur matchen soll wenn vor ( **lookbehind**) oder nach (**lookahead**) einem Match etwas bestimmtes folgen oder nicht folgen darf. Die Anleitung dazu findet man leicht mit Hilfe von ChatGPT ("gib mir ein Beispiel für positiven und negativen Regex Lookahead und Lookbehind im Kontext eines Python-Programms") oder Google ("regex lookahead lookbehind tutorial").

Aber in diesem Fall reicht es tatsächlich, wenn wir das ganze als Teil des Matches betrachten.

In [26]:
# beachte das ([^0-9,.]|$) am Ende, es bedeutet "entweder keine Zahl, Komma oder Punkt, oder das Ende des Texts"
# und das ([^0-9,.]|^)([0-9]+) am Anfang bedeutet "entweder keine Zahl, Komma oder Punkt, oder der Anfang des Texts"
komma_muster = r"([^0-9,.]|^)([0-9]+)(\.[0-9]{3,3})*(\,[0-9]{1,2})?([^0-9,.]|$)"



In [27]:
if not re.search(komma_muster, '100,000.5 $'):
  print("Kein Match, so soll es sein!")

Kein Match, so soll es sein!


In [28]:
if re.search(komma_muster, '1.130,5 EUR'):
  print("Ein Match, so soll es sein!")

Ein Match, so soll es sein!


In [29]:
if re.search(komma_muster, '1.130 EUR'):
  print("Ein Match, so soll es sein!")

Ein Match, so soll es sein!


Jetzt haben wir die Spalte i

